In [ ]:
# import libraries
from __future__ import print_function

import sys
import pdb

import numpy as np
import pandas as pd
import math

from pandas import Series, DataFrame
import matplotlib.pyplot as plt

from pylab import rcParams
#import seaborn as sb

from numpy.random import randn
import glob
from math import sqrt

#-------- widgets
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual

#--------time data manipulation
import time
from time import strptime
# -----------plotly for interactive plotting
#import cufflinks as cf
# import plotly

# plotly.tools.set_credentials_file(username='tonylee2016', api_key='2MJDXLESXkHo9EtK9ZJR')
#from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

import plotly as py
import plotly.graph_objs as go
py.__version__
py.offline.init_notebook_mode(connected=True)

from scipy.integrate import solve_ivp,simps

In [ ]:
# ode fcns
def RK4(f,x0,v_t,v_u = np.array([])):
    
    t_size = len(v_t)
    x_size = len(x0)
    #pdb.set_trace()
    u_size = len(v_u)
        
    v_x = np.zeros((t_size,x_size))
    
    #assign the intial value 
    v_x[0,:] = x = x0
    t = v_t[0]
    u = 0
    
    for i in range(1,t_size):
        
        
        #x[i-1],t[i-1]
        
        if  u_size > 1: 
            try: 
                u = v_u[i,:]
            except:
                u = v_u[i]            
            
        dt = v_t[i] - v_t[i - 1]
        
        k1 = dt * f(t,x,u)
        k2 = dt * f(t + 0.5*dt, x + 0.5*k1,u)
        k3 = dt * f(t + 0.5*dt, x + 0.5*k2,u)
        k4 = dt * f(t + dt, x + k3,u)
        
        #x[i],t[i]
#        pdb.set_trace()
        
        t += dt
        x += (k1 + k2 + k2 + k3 + k3 + k4) / 6
        
        v_x[i,:] = x
    
    return v_x,v_t
   

In [ ]:
#testing for the ode solver
def f1(t,x,u = None):
    dx1 = -0.01*x[0]
    dx2 = 1
#    pdb.set_trace()
    return np.array([dx1,dx2])
#def RK4(f,x0,v_t,v_u):
[xs,ts] = RK4(f1, [100,0], np.array([0,1,2,3,4,5,6,7,8,9,10,11,12,13,15,18,20,30,50])*10)
plt.figure(1)
plt.clf()
#plt.plot(ts, .transpose(), 'k', label='theory')
plt.plot(ts, xs[:,0], 'k--', label='Output')
plt.legend(loc='best')

In [ ]:
# libraries

'''
comman functions
'''
def plotdf(df,Selects):
    py.offline.iplot([{
            'x': df.Time,
            'y': df[col],
            'name': col
        }  for col in Selects], filename='basic-line')

'''
basic classes
'''
class soc_ocv_curve:
    def __init__(self,soc,ocv):
        self.ocv = ocv
        self.soc = soc
    def disp(self):
        plt.xlabel('soc')
        plt.ylabel('ocv')
        plt.plot(self.soc,self.ocv)
    def lookup(self,soc_in):
        return np.interp(soc_in,self.soc,self.ocv)

class component:
    def __init__(self,name):
        self.name = name
    def rename(self,newname):
        self.name = newname

#dynamical sytem
# class Lineardynamic():
#     def __init__(self,x_size,y_size,u_size):
#         self.x = np.zeros(x_size)
#         self.dx = np.zeros(x_size)
#         self.y = np.zeros(y_size)
#         self.u = np.zeros(u_size)
#         self.A = np.zeros(x_size,x_size)
#         self.B = np.zeros(x_size,u_size)
#         self.C = np.zeros(y_size,x_size)
#
#     def refresh():
#       self.dx = np.matmul(self.A,self.x) + np.matmul(self.B,self.y)
#     def run(ics,ts):

'''
component classes
'''

import re
from asammdf import MDF, Signal

class BatteryData(component):

    def __init__(self,name):
        component.__init__(self,name)

    def read(self,filename):
        if "csv" in filename:
            print("imported " + filename + ' for ' + self.name)
            self.df = pd.read_csv(filename)
        else:
            print(filename + " is illegal")

    def clean(self):
        if hasattr(self,'df'):
            self.df.Time_ms /=1000
            self.df.rename(columns = {"Time_ms":"Time"},inplace=True)
            self.Current = data1.df.Current.values
            self.Time = data1.df.Time.values
            print("cleaned " + self.name)
    #       df.rename(columns = lambda x: re.sub('^.*Voltage.*$','Voltage',x))
    #       df.rename(columns = lambda x: re.sub('^.*Current.*$','Current',x))

    def plot(self):
        plotdf(self.df,["Voltage","Current"])
class BatterySol(component):
    def __init__(self,vol,xs,ts,v_u,modelName,dataName,name):
        component.__init__(self,name)
        self.Current = v_u
        self.States = xs
        self.Voltage = vol
        self.Time    = ts
        self.modelName = modelName
        self.dataName = dataName
        self.score = 0
        
    def cmpData(BatteryData):
        print("compare the solution for \n"
              + "model: " + self.modelName + " and data " + self.dataName
              + "with data " + BatteryData.Name)
        
class BatteryModel(component):
    def __init__(self,RCs,ocv_curve,name):
        component.__init__(self,name)
        self.RCs = RCs
        self.ocv_curve = ocv_curve

    def disp(self):
        VAR_NAMES = ["R0", "R1", "C1", "R2", "C2", "C"]
        for i in range(0,6):
            print(VAR_NAMES[i] + " = " + str(self.RCs[i]))
        self.ocv_curve.disp()

    def update(self,RCs):
        self.RCs = RCs

    def run(self,t,x,i0):
        dxdt0 = 1/self.RCs[2]*(i0 - 1/self.RCs[1]*x[0])
        dxdt1 = 1/self.RCs[4]*(i0 - 1/self.RCs[3]*x[1])
        dxdt2 = -1/self.RCs[-1]*i0
        dxdt = np.array([dxdt0,dxdt1,dxdt2])
        return dxdt

   # def dydt():
'''
top classes
'''
class BatteryLearn:

    def __init__(self,name):
        self.name = name
        self.models = []
        self.datas  = []
        self.sols   = []

    def pushdata(self,BatteryData):
        print("pushed " + BatteryData.name+" to " + self.name)
        self.datas.append(BatteryData)

    def pushmodel(self,Batterymodel):
        print("pushed " + Batterymodel.name + " to " + self.name)
        self.models.append(Batterymodel)

    def clearAll(self):
        self.models = []
        self.datas  = []

    def cleanData(self):
        for data in self.datas:
            self.data.clean()

    def runModel(self,model_i,data_i,x0,tspan = None):

        if not tspan:
            ts = self.datas[data_i].df.Time.values
        else:
            ts = self.datas[data_i].df.Time.values[tspan[0]:tspan[1]]
        
        v_u = self.datas[data_i].df.Current.values
        [xs,ts] = RK4(self.models[model_i].run,x0,ts,v_u)
        Voltage = self.models[model_i].ocv_curve.lookup(xs[:,2]) + xs[:,0] + xs[:,1]
        
        return Voltage,xs,ts,v_u,self.models[model_i].name,self.datas[data_i].name
    
        
#        sol = solve_ivp(lambda t, y: self.models[model_i].run(t, y,self.datas[data_i].i0),\
#                        [ts[0],ts[-1]], \
#                        y0, \
#                        method='RK45',\
#                        t_eval= ts,\
#                        first_step = 1,\
#                        max_step = 1 )
        
        #self.sols.append(sol)
        print("run finished")


\begin{align}
\dot{v}_1& = \frac{1}{C_1}(i_0 -\frac{v_1}{r_1}) \\
\dot{v}_2& = \frac{1}{C_2}(i_0 -\frac{v_2}{r_2}) \\
\dot{z}  &= - \frac{1}{Q}{i_0}
\end{align}

In [ ]:
# set up testing for battery data
import sys
import os
import numpy as np

#module_path = (os.getcwd()+"\lib")
#if module_path not in sys.path:
#    sys.path.append(module_path)

#from BatteryLearn import soc_ocv_curve, BatteryModel,BatteryData,BatteryLearn

data1 = BatteryData("trial")
#filepath = []
#for i in range(1,5):
#filepath += [os.getcwd()+"\\TestResults_SOCOCV\\A0" + str(i) + ".csv"]
filepath = os.getcwd()+"\\TestResults_SOCOCV\\A01.csv"
data1.read(filepath)
data1.clean()

In [ ]:
data2.mdf

In [ ]:
# model parameters
#VAR_NAMES = ["R0", "R1", "C1", "R2", "C2", "C"]
Rcs = [0.1,1,100,2,300,100]
ocv = [3, 3.1, 3.2, 3.3, 3.4, 3.5]
soc = [0, 20,  40,  60,  80,  100]
curve1 = soc_ocv_curve(soc,ocv)
model1 = BatteryModel(Rcs,curve1,"model1")
#model1.disp()

In [ ]:
learn1 = BatteryLearn("learn1")
learn1.name
learn1.pushdata(data1)
learn1.pushmodel(model1)

In [ ]:
#def runModel(self,model_i,data_i,x0,tspan = None,v_u = None):
a = learn1.runModel(0, 0,[0,0,0])

In [ ]:
sol1 = BatterySol(*a) 

In [ ]:
plt.plot(ts, vol - data1.df.Voltage.values, 'k-', label='Input')
#plt.plot(sol.t, sol.y[0], 'k--', label='Output')

In [ ]:
learn1.datas[0].Time

In [ ]:
# %% Define independent function and derivative function

class b:
    def __init__(self):
        pass
    def f(self,t, y,x):
        dydt = x(t) - y / 4
        return dydt

class a:
    def __init__(self):
        pass
    def callback(self,t):
        print(t)
        return np.cos(3 * t)

# %% Define time spans, initial values, and constants
tspan = np.linspace(0, 15, 1000)
yinit = [-1]
a1 = a()
b1 = b()
# %% Solve differential equation
sol = solve_ivp(lambda t, y: b1.f(t, y,a1.callback),
                [tspan[0], tspan[-1]], yinit, t_eval=tspan)

# %% Plot independent and dependent variable
# Note that sol.y[0] is needed to extract a 1-D array
plt.figure(1)
plt.clf()
plt.plot(sol.t, callback(sol.t), 'k-', label='Input')
plt.plot(sol.t, sol.y[0], 'k--', label='Output')
plt.legend(loc='best')

In [ ]:
a  =[1,2,3]
a[0]
a[a[0]:a[1]]